### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ

### データ
- mushroomの50サンプル分

### yのサンプリング
- 条件に合うまでサンプルを作り続ける

### 改訂(2020/08/03)
- ori_tiのkeyがなかった場合に1とする必要がない
- sum([j[1] for j in ti_list])は必ず100である
- 正確な分布を作ることが必要なので、yを全列挙する必要がある。

### 改訂（2020/08/18）
- mushroom_l データを更新（sumが8になるようにした）

### 改訂（2020/08/20）
- make_y関数のバグを発見



In [61]:
#!python3 --version

In [62]:
#!pip3 install dwave-neal

In [63]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools

%matplotlib inline
import matplotlib.pyplot as plt

In [64]:
#X_mushroom = pd.read_csv("../input/SNP_df1010.csv", sep=',', index_col=0).iloc[:10, :10]
#y_mushroom = pd.read_csv('../input/y10.csv', sep=',', index_col=0)['y'][:10]

X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0)
y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

j = '4'

#X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0)
#y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

-----

In [65]:
def make_y(j):
    y_list = []
    y_list_all = itertools.product('01', repeat=num_samples)

    for y_tuple in y_list_all:
        y_list_list = [int(n) for n in list(y_tuple)]
        y = pd.Series(y_list_list)

        if sum(y) - y_sum != 0:
            continue

        for i in list(j):
            if np.dot(X_mushroom[i], y) - np.dot(X_mushroom[i], y_mushroom) == 0:
                y_list.append(y) #pd.Seriesを要素として足している
    return y_list

In [66]:
minP_cols = []
servey_num_col_list = [jj for jj in columns if jj != j]

y_list = make_y(j)
print(len(y_list)) ##

p = {}
for i in servey_num_col_list:
    ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

    ti_dic = {} #{内積の値:該当数}という辞書。

    for y in y_list:       
        if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
            ti_dic[int(np.dot(X_mushroom[i], y))] += 1
        else:
            ti_dic[int(np.dot(X_mushroom[i], y))] = 1

    count_better_t = sum([v for k, v in ti_dic.items() if k >= ori_ti]) 
    #count_better_t = sum([v for k, v in ti_dic.items() if k > ori_ti]) #2020/08/20変更
    p[i] = count_better_t / len(y_list)

minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
print(minP_keys) ##

minP_cols += minP_keys
servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]
print(servey_num_col_list)

560
['1', '27']
['0', '2', '3', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118']


In [75]:
len(y_list)

560

In [77]:
print(p['1'])
print(p['1']*len(y_list))

0.0017857142857142857
1.0


In [78]:
print(p['2'])
print(p['2']*len(y_list))

0.7857142857142857
440.0


In [79]:
print(p['26'])
print(p['26']*len(y_list))

1.0
560.0


In [80]:
print(p['27'])
print(p['27']*len(y_list))

0.0017857142857142857
1.0


In [57]:
#≧を＞にした結果

minP_cols = []
servey_num_col_list = [jj for jj in columns if jj != j]

y_list = make_y(j)
print(len(y_list)) ##

p = {}
for i in servey_num_col_list:
    ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

    ti_dic = {} #{内積の値:該当数}という辞書。

    for y in y_list:       
        if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
            ti_dic[int(np.dot(X_mushroom[i], y))] += 1
        else:
            ti_dic[int(np.dot(X_mushroom[i], y))] = 1

    count_better_t = sum([v for k, v in ti_dic.items() if k > ori_ti]) #2020/08/20変更
    p[i] = count_better_t / len(y_list)

minP_keys = [k for k, v in p.items() if v == min(list(p.values()))] 
print(minP_keys) ##ここの結果が変わった（≧だと1,27しか出力されなかった）

minP_cols += minP_keys
servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]
print(servey_num_col_list)

560
['1', '7', '10', '11', '15', '16', '17', '19', '22', '23', '25', '27', '29', '31', '32', '33', '34', '36', '37', '38', '41', '42', '44', '45', '46', '48', '49', '51', '54', '57', '58', '60', '62', '64', '65', '67', '70', '73', '74', '76', '79', '80', '81', '82', '83', '84', '85', '88', '89', '91', '95', '96', '97', '100', '101', '102', '103', '105', '106', '108', '115', '118']
['0', '2', '3', '5', '6', '8', '9', '12', '13', '14', '18', '20', '21', '24', '26', '28', '30', '35', '39', '40', '43', '47', '50', '52', '53', '55', '56', '59', '61', '63', '66', '68', '69', '71', '72', '75', '77', '78', '86', '87', '90', '92', '93', '94', '98', '99', '104', '107', '109', '110', '111', '112', '113', '114', '116', '117']


In [81]:
enul_p = pd.DataFrame.from_dict(p, orient='index').T
print(enul_p)

enul_p.to_csv('../output/mushrooml/enul_p_conf4.csv')

     0         1         2    3         5     6    7    8         9   10  ...  \
0  1.0  0.001786  0.785714  1.0  0.607143  0.35  1.0  0.9  0.785714  1.0  ...   

   109  110  111       112   113  114  115  116  117     118  
0  1.0  0.9  1.0  0.785714  0.35  1.0  1.0  1.0  1.0  0.1875  

[1 rows x 118 columns]


-----

### t^0のみ

In [ ]:
#t^0に当てはまるyの数を知る
y_list = []
y_list_all = itertools.product('01', repeat=num_samples)

for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum == 0:
        y_list.append(y) #type(y)はSeries
len(y_list)

In [ ]:
ori_t1 = int(np.dot(X_mushroom['1'], y_mushroom)) #特徴量iを表すp
#print(ori_t1) #8
for y in y_list:
    #print(y)
    if list(y)==list(y_mushroom):
        print(y)
    if int(np.dot(X_mushroom['1'], y))==ori_t1:
        print(y)
        print('---------')#originalのyしか該当するyはないようだ

In [ ]:
0.000078*12870  #約1(1.00386)

In [ ]:
plt.rcParams["figure.figsize"] = [100, 150]  # 図の縦横のサイズ([横(inch),縦(inch)])
plt.rcParams["figure.autolayout"] = True   # レイアウトの自動調整を利用するかどうか
plt.rcParams["figure.subplot.wspace"] = 0.10# 図が複数枚ある時の左右との余白
plt.rcParams["figure.subplot.hspace"] = 0.10# 図が複数枚ある時の上下との余白

fig, ax = plt.subplots(int(num_cols/3)+1, 3)

minP_cols = []
servey_num_col_list = columns

enul_p = {}
for index, i in enumerate(servey_num_col_list):
    
    ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

    ti_dic = {} #{内積の値:該当数}という辞書。

    for y in y_list:       
        if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
            ti_dic[int(np.dot(X_mushroom[i], y))] += 1
        else:
            ti_dic[int(np.dot(X_mushroom[i], y))] = 1

    ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
    franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
    enul_p[i] = franc/len(y_list)
    
    n_row = int(index%(int(num_cols/3)+1))#行
    
    #列
    if index < int(num_cols/3)+1:
        n_col = 0
    elif index < (int(num_cols/3)+1)*2:
        n_col = 1
    else:
        n_col = 2
    
    #ヒストグラムを出力
    ax[n_row, n_col].set_title(i, fontsize=15)
    ax[n_row, n_col].bar([j[0] for j in ti_list], [j[1] for j in ti_list])
    ax[n_row, n_col].scatter(ori_ti, 100, s=200, marker='D')
    ax[n_row, n_col].set_xlabel('inner product size', fontsize=10)
    ax[n_row, n_col].set_ylabel('number', rotation=90, fontsize=10, labelpad=20)
    
#図が重ならないようにする
fig.tight_layout()

fig.subplots_adjust(bottom=0, left=0, top=0.1, right=0.1, wspace=0)
    
#全体に一つの題名を表示
fig.suptitle('inner product size and its number using all y which meet conditions', fontsize=15)

#図を表示
#plt.show()

#fig.savefig('../output/mushrooml/enul_p.png')

enul_p = pd.DataFrame.from_dict(enul_p, orient='index').T
print(enul_p)

enul_p.to_csv('../output/mushrooml/enul_p.csv')

In [ ]:
enul_p

In [ ]:
#enul_pにおける各列の最小さい値
enul_p.idxmin(axis=1)

In [ ]:
np.dot(X_mushroom['1'], y_mushroom)

In [ ]:
y_mushroom

In [ ]:
np.dot(X_mushroom['1'], pd.Series([0, 1,1,1,0,1,0,1,0,1,0,1,0, 0, 0, 1])) 

In [ ]:
np.dot(X_mushroom['27'], y) - np.dot(X_mushroom['27'], y_mushroom)

In [ ]:
#選ばれるyがあるかどうかの確認（p値とは無関係）
y_list = []
y_list_all = itertools.product('01', repeat=num_samples)

for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum != 0:
        continue

    if np.dot(X_mushroom['1'], y) - np.dot(X_mushroom['1'], y_mushroom) == 0:
        if np.dot(X_mushroom['27'], y) - np.dot(X_mushroom['27'], y_mushroom) == 0:
            print(y)

In [ ]:
#選ばれるyがあるかどうかの確認（p値とは無関係）
y_list = []
y_list_all = itertools.product('01', repeat=num_samples)

for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum != 0:
        continue

    if np.dot(X_mushroom['1'], y) - np.dot(X_mushroom['1'], y_mushroom) == 0:
        print(y)